# Idea
The idea is that we predict rotated linear dimension position from lines and text from the drawing.

# Thoughts
Basic logic is we split dataset to input lines or texts positions StartPoint, EndPoint, Position XYZ and predict dimension extension line poistion XLine1Point, XLine2Point XYZ.

We going to group samples by FileId. That is each sample will contain variable length data (attributes of variable number of  lines and text) and variable output data (variable number of dimensions).

I intend to [use RNN for it](https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79)

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
# https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/
import torch
from torch import nn
from torch.utils.data import Dataset, SubsetRandomSampler
from torch.nn import functional as F

device = torch.device("cpu")

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.empty_cache()    

In [4]:
device

device(type='cuda')

# Dataset implementation

In [5]:
from dataset import DwgDataset
batch_size = 1

dwg_dataset = DwgDataset(pickle_file = 'test_dataset.pickle', batch_size = batch_size)

train_loader = dwg_dataset.train_loader
val_loader   = dwg_dataset.val_loader
test_loader  = dwg_dataset.test_loader

ent_features = dwg_dataset.entities.ent_features
dim_features = dwg_dataset.entities.dim_features

# Model and training


In [6]:
from model import RnnDecoder, RnnEncoder
rnn_encoder = RnnEncoder(ent_features, 1024, enforced_device = device).to(device)
rnn_decoder = RnnDecoder(1024, dim_features, enforced_device = device).to(device)

In [7]:
from train import train_model, plot_history

In [8]:
lr = 11
epochs = 30
decoder_optimizer = torch.optim.Adam(rnn_decoder.parameters(), lr = lr)
encoder_optimizer = torch.optim.Adam(rnn_encoder.parameters(), lr = lr)
from chamfer_distance_loss import MyChamferDistance
loss = MyChamferDistance()

loss_history, train_history, val_history = train_model(
    encoder = rnn_encoder, 
    decoder = rnn_decoder, 
    train_loader = train_loader,
    val_loader = val_loader,
    loss = loss,
    decoder_opt = decoder_optimizer,
    encoder_opt = encoder_optimizer,
    epochs = epochs)

plot_history(loss_history, train_history, val_history)

[0-0 @ 0.2 sec] Loss: 3.762907 Train err: 46.6%
[0-1 @ 0.2 sec] Loss: 3.569414 Train err: 57.2%
[0-2 @ 0.2 sec] Loss: 3.572269 Train err: 57.7%
[0-3 @ 0.3 sec] Loss: 3.913982 Train err: 100.0%
[0-4 @ 0.4 sec] Loss: 4.578840 Train err: 100.0%
[0-5 @ 0.5 sec] Loss: 3.559657 Train err: 56.0%
[0-6 @ 0.8 sec] Loss: 3.851068 Train err: 52.6%
[0-7 @ 1.0 sec] Loss: 3.564227 Train err: 46.9%
[0-8 @ 1.1 sec] Loss: 4.251142 Train err: 68.7%
[0-9 @ 1.1 sec] Loss: 3.637224 Train err: 43.9%
[0-10 @ 1.7 sec] Loss: 3.505424 Train err: 57.5%
[0-11 @ 1.8 sec] Loss: inf Train err: 100.0%
[0-12 @ 1.9 sec] Loss: inf Train err: 100.0%
[0-13 @ 1.9 sec] Loss: 3.450802 Train err: 52.9%
[0-14 @ 3.5 sec] Loss: 4.588172 Train err: 100.0%
[0-15 @ 3.6 sec] Loss: 3.780910 Train err: 39.1%
[0-16 @ 3.6 sec] Loss: inf Train err: 100.0%
[0-17 @ 3.7 sec] Loss: 3.072478 Train err: 100.0%
[0-18 @ 3.7 sec] Loss: inf Train err: 100.0%
[0-19 @ 4.1 sec] Loss: 4.088762 Train err: 50.8%
[0-20 @ 4.2 sec] Loss: inf Train err: 100.

RuntimeError: CUDA out of memory. Tried to allocate 116.00 MiB (GPU 0; 4.00 GiB total capacity; 2.79 GiB already allocated; 62.61 MiB free; 2.85 GiB reserved in total by PyTorch)

In [9]:
# https://blog.paperspace.com/pytorch-memory-multi-gpu-debugging/
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 86% |


In [ ]:
from train import calculate_accuracy
i = 0
for (x, y) in iter(val_loader):
    outs, learned = rnn_encoder(x)
    decoded = rnn_decoder(outs, learned)
    
    yyy = []
    for yy in y:
        yyy.append(yy.shape[0])
    ppp = []
    for dd in decoded:
        ppp.append(dd.shape[0])
    
    print('actual:', yyy)
    print('predicted:', ppp)
    
    lv = loss(decoded, y)
    print ('loss:', lv)

    acc = calculate_accuracy(decoded, y)
    print('accuracy:', acc)

    i += 1
    print(i, '------------------------------')

In [ ]:
ii = pd.DataFrame(x[0].cpu().detach().numpy())
print(ii.head())
yy = pd.DataFrame(y[0].cpu().detach().numpy())
print(len(yy))
print(yy.head())
pp = pd.DataFrame(decoded[0].cpu().detach().numpy())
print(len(pp))
print(pp.head())

In [ ]:
from train import calculate_accuracy
torch.cuda.empty_cache()

rnn_encoder.eval()
rnn_decoder.eval()

test_accuracies = []
for (x,y) in test_loader:
    with torch.no_grad():
        out, hidden = rnn_encoder(x)
        prediction = rnn_decoder(out, hidden)
        accuracy = calculate_accuracy(prediction, y)
        test_accuracies.append(accuracy)
        
mean_test_accuracy = np.mean(test_accuracies)
print('Accuracy on testing: {0:2.3f}'.format(mean_test_accuracy))